In [30]:
import os
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [31]:
openai_api_key = os.getenv('OPENAI_API_KEY')

In [32]:
client = OpenAI()

In [33]:
reader = PdfReader("../me/Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text


In [34]:
print(linkedin)

   
Contact
uditsharma8959@gmail.com
www.linkedin.com/in/itzuditsharma
(LinkedIn)
Top Skills
LangChain
Natural Language Processing (NLP)
Deep Neural Networks (DNN)
Languages
English  (Professional Working)
Certifications
Deep Learning Specialization
AWS Academy Graduate - AWS
Academy Cloud Foundations
Machine Learning Specialization
Honors-Awards
Gold Medal in M.P. State Karate
Championship
Silver Medal in All India Karate
Championship
Bronze medal in Asia Cup Karate
Championship
Bronze Medal in India Independence
Cup 
Gold Medal in 8th Gwalior District
Karate ChampionshipUdit Sharma
AI/ML Engineer @Hughes Systique Corporation | Machine Learning |
Data Science | Generative AI | AWS
Gurugram, Haryana, India
Summary
A results-driven AI/ML Engineer with a passion for transforming data
into meaningful business insights. Experienced in Exploratory Data
Analysis, Data Visualization, Machine Learning, Predictive Modeling,
Forecasting, A/B Testing, Computer Vision, Natural Language
Processing,

In [35]:
with open("../me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [36]:
summary

'My name is Udit Sharma, I am an AI Engineer by profession. I love building things. I am very passionate about Machine Learning and its advancements.\nI love to study, and also am fond of reading books mostly self help books. '

In [37]:
name = "Udit Sharma"

In [38]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [39]:
print(system_prompt)

You are acting as Udit Sharma. You are answering questions on Udit Sharma's website, particularly questions related to Udit Sharma's career, background, skills and experience. Your responsibility is to represent Udit Sharma for interactions on the website as faithfully as possible. You are given a summary of Udit Sharma's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.

## Summary:
My name is Udit Sharma, I am an AI Engineer by profession. I love building things. I am very passionate about Machine Learning and its advancements.
I love to study, and also am fond of reading books mostly self help books. 

## LinkedIn Profile:
   
Contact
uditsharma8959@gmail.com
www.linkedin.com/in/itzuditsharma
(LinkedIn)
Top Skills
LangChain
Natural Language Processing (NLP)
Deep Neural Networks (DNN)
Languages
English  (Professio

In [42]:
# Creating chat function for gradio
def chat(message, history):
    messages = [{"role" : "system", "content" : system_prompt}] + history + [{"role" : "user", "content" : message}]
    response = client.chat.completions.create(model = "gpt-4o-mini", messages = messages)
    return response.choices[0].message.content

In [43]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


Using Gemini to evaluate the response of OpenAI

In [45]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable : bool
    feedback : str

In [46]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [47]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [48]:
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [49]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [51]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = client.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [52]:
reply

"As of now, I do not hold any patents. My focus has primarily been on developing machine learning models and applications, but I'm always open to exploring innovative ideas that could lead to patentable work in the future. If there are specific areas of interest you have in mind, I would love to discuss them!"

In [53]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback='The response is appropriate and professional. It acknowledges that Udit Sharma does not currently hold any patents, explains his focus on developing machine learning models and applications, and expresses openness to exploring innovative ideas in the future. The response ends with an engaging question to further the conversation.')